# Querying Data Modeling

This tutorial demonstrates how to query a data model in Cognite Data Fuction (CDF) with Python 

**Prerequisites**

* Installed `cognite-sdk`
* Setup a service principal to connect to CDF.
* Use `cognite-toolkit` quickstart to setup configuration and populated the `CogniteCore` Model [Toolkit quickstart](https://docs.cognite.com/cdf/deploy/cdf_toolkit/references/packages/quickstart)

## Setting up a cognite client

In [1]:
import os

from cognite.client import CogniteClient
from dotenv import load_dotenv

load_dotenv("../../.env")

client = CogniteClient.default_oauth_client_credentials(
    project=os.environ["CDF_PROJECT"],
    cdf_cluster=os.environ["CDF_CLUSTER"],
    tenant_id=os.environ["IDP_TENANT_ID"],
    client_id=os.environ["IDP_CLIENT_ID"],
    client_secret=os.environ["IDP_CLIENT_SECRET"],
)

## Querying across a direct relation

First, we will query across a direct relation. For example, we can get the `parent` starting from `CogniteAsset`

In [2]:
from cognite.client import data_modeling as dm
from cognite.client.data_classes import filters

asset_view = dm.ViewId("cdf_cdm", "CogniteAsset", "v1")
# The instance space with the instance demo data
data_space = "springfield_instances"

In [3]:
query = dm.query.Query(
    with_={
        "asset": dm.query.NodeResultSetExpression(
            limit=10,
            filter=filters.And(filters.HasData(views=[asset_view]), filters.Equals(["node", "space"], data_space)),
        ),
        "parent": dm.query.NodeResultSetExpression(
            from_="asset", through=dm.PropertyId(source=asset_view, property="parent")
        ),
    },
    select={
        "asset": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
        "parent": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
    },
)

result = client.data_modeling.instances.query(query)

In [4]:
df = result["asset"].to_pandas(expand_properties=True)
df

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,pathLastUpdatedTime,name,description,sourceId,sourceUpdatedUser,parent
0,springfield_instances,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:20:20.316,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2ND STAGE COMP DRY GAS SEAL SYS ON PH,682546,8137,NaN
1,springfield_instances,WMT:230900,4,2024-11-13 16:30:39.129,2024-11-08 14:20:20.316,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,230900,GAS COMPRESSION AND RE-INJECTION ON PH,681760,8137,NaN
2,springfield_instances,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:20:27.925,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,682540,8137,NaN
3,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:27.925,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,681794,8137,"{'space': 'springfield_instances', 'externalId..."
4,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:28.103,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,681825,8137,"{'space': 'springfield_instances', 'externalId..."
5,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,681824,8137,"{'space': 'springfield_instances', 'externalId..."
6,springfield_instances,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMP LUBE OIL SYS-PH,1ST STAGE COMP LUBE OIL SYS ON PH,682544,8137,NaN
7,springfield_instances,WMT:23-1ST STAGE COMP ENCLOSURE-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMP ENCLOSURE-PH,1ST STAGE COMP ENCLOSURE ON PH,682542,8137,NaN
8,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,681792,8137,"{'space': 'springfield_instances', 'externalId..."


In [5]:
result["parent"]

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,pathLastUpdatedTime,name,description,sourceId,sourceUpdatedUser,parent
0,springfield_instances,WMT:230900,4,2024-11-13 16:30:39.129,2024-11-08 14:20:20.316,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,230900,GAS COMPRESSION AND RE-INJECTION ON PH,681760,8137,NaN
1,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:27.925,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,681794,8137,"{'space': 'springfield_instances', 'externalId..."
2,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,681792,8137,"{'space': 'springfield_instances', 'externalId..."


## Quering across reverse direct relation of single relation

We can for example query across the `children` in `CogniteAsset`. The `children` property is a reverse direct relation of `parent`, so we query in `inwards` direction towards parent

In [6]:
query = dm.query.Query(
    with_={
        "asset": dm.query.NodeResultSetExpression(
            limit=10,
            filter=filters.And(filters.HasData(views=[asset_view]), filters.Equals(["node", "space"], data_space)),
        ),
        "children": dm.query.NodeResultSetExpression(
            from_="asset", through=dm.PropertyId(source=asset_view, property="parent"), direction="inwards"
        ),
    },
    select={
        "asset": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
        "children": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
    },
)

result = client.data_modeling.instances.query(query)

In [7]:
result["asset"]

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,pathLastUpdatedTime,name,description,sourceId,sourceUpdatedUser,parent
0,springfield_instances,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:20:20.316,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2ND STAGE COMP DRY GAS SEAL SYS ON PH,682546,8137,NaN
1,springfield_instances,WMT:230900,4,2024-11-13 16:30:39.129,2024-11-08 14:20:20.316,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,230900,GAS COMPRESSION AND RE-INJECTION ON PH,681760,8137,NaN
2,springfield_instances,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:20:27.925,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,682540,8137,NaN
3,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:27.925,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,681794,8137,"{'space': 'springfield_instances', 'externalId..."
4,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:28.103,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,681825,8137,"{'space': 'springfield_instances', 'externalId..."
5,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,681824,8137,"{'space': 'springfield_instances', 'externalId..."
6,springfield_instances,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMP LUBE OIL SYS-PH,1ST STAGE COMP LUBE OIL SYS ON PH,682544,8137,NaN
7,springfield_instances,WMT:23-1ST STAGE COMP ENCLOSURE-PH,3,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMP ENCLOSURE-PH,1ST STAGE COMP ENCLOSURE ON PH,682542,8137,NaN
8,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,681792,8137,"{'space': 'springfield_instances', 'externalId..."


In [8]:
result["children"]

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,parent,pathLastUpdatedTime,name,description,sourceId,sourceUpdatedUser
0,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:27.925,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,681794,8137
1,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:20:28.103,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,681825,8137
2,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,681824,8137
3,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,5,2024-11-13 16:30:39.129,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...","{'space': 'springfield_instances', 'externalId...",2024-11-13T16:30:39.129624+00:00,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,681792,8137


## Querying across reverse direct relation of listable relation

We want all activities connected to equipment. The property `activities` in `CogniteEquipment` is a reverse direct relation of `equipment` in `CogniteActivity`.  
The `/query` enpoint does not support querying accross a reverse direct relation of listable relation. Instead, we will first lookup `CogniteEquipment` and the after we have retrieved the equipment items,
we will use `/search` to lookup activities that are connected to the equipment we found.

In [9]:
equipment_view = dm.ViewId("cdf_cdm", "CogniteEquipment", "v1")
activity_view = dm.ViewId("cdf_cdm", "CogniteActivity", "v1")

In [10]:
equipment = client.data_modeling.instances.list(limit=100, sources=equipment_view, space=data_space)

In [11]:
is_equipment = filters.In(activity_view.as_property_ref("equipment"), [item.as_id() for item in equipment])

In [12]:
activities = client.data_modeling.instances.search(activity_view, filter=is_equipment, limit=1_000)

In [13]:
activities

,space,external_id,version,last_updated_time,created_time,instance_type,assets,equipment,timeSeries,name,tags
0,springfield_instances,6834296454446862,8,2024-11-11 04:01:24.673,2024-11-08 14:20:32.010,node,[],"[{'space': 'springfield_instances', 'externalI...","[{'space': 'springfield_instances', 'externalI...",Ettersyn av instrumentering,[23-PT-96186-03]
1,springfield_instances,4779071375364741,8,2024-11-11 04:01:24.673,2024-11-08 14:20:32.968,node,[],"[{'space': 'springfield_instances', 'externalI...","[{'space': 'springfield_instances', 'externalI...",Ettersyn av instrumentering,[23-PT-96186-03]
2,springfield_instances,7676140200338059,8,2024-11-11 04:01:24.673,2024-11-08 14:21:09.459,node,[],"[{'space': 'springfield_instances', 'externalI...","[{'space': 'springfield_instances', 'externalI...",Årlig ettersyn av instrumentering,[23-PT-96186-03]
3,springfield_instances,2419388136472507,6,2024-11-10 04:01:27.838,2024-11-08 14:20:42.833,node,[],"[{'space': 'springfield_instances', 'externalI...","[{'space': 'springfield_instances', 'externalI...",,[23-PDT-96167]
4,springfield_instances,3625599615521242,6,2024-11-10 04:01:27.838,2024-11-08 14:20:42.833,node,[],"[{'space': 'springfield_instances', 'externalI...","[{'space': 'springfield_instances', 'externalI...",,[23-PDT-96167]
...,...,...,...,...,...,...,...,...,...,...,...
995,springfield_instances,3615479516814565,3,2024-11-09 04:01:18.514,2024-11-08 14:20:49.286,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig ettersyn av instrumentering,[23-TE-96103-01]
996,springfield_instances,2539613360803803,3,2024-11-09 04:01:18.514,2024-11-08 14:20:49.286,node,[],"[{'space': 'springfield_instances', 'externalI...",[],*** / ***,[23-TE-96103-01]
997,springfield_instances,8028091942242686,3,2024-11-09 04:01:18.514,2024-11-08 14:20:50.755,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig sampling assessment,[23-LT-96182-01]
998,springfield_instances,7618946439648778,3,2024-11-09 04:01:18.514,2024-11-08 14:20:52.102,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig ettersyn av instrumentering,[23-TE-96103-01]


In [14]:
equipment

,space,external_id,version,last_updated_time,created_time,instance_type,name,description,files,manufacturer,serialNumber,sourceId,sourceUpdatedUser,asset
0,springfield_instances,WMT:23-YAH-96119-01,1,2024-11-08 14:20:20.365,2024-11-08 14:20:20.365,node,23-YAH-96119-01,VRD - PH 1STSTGGEAR 2 JOURNBRG NDE : VIBRATION...,[],1686,1000000000293659,700491,1001,NaN
1,springfield_instances,WMT:23-TAH-96113-03,1,2024-11-08 14:20:20.365,2024-11-08 14:20:20.365,node,23-TAH-96113-03,VRD - PH 1STSTGGEAR 2 JOURNBRG NDE : TEMPERATU...,[],1686,1000000000284937,696113,1001,NaN
2,springfield_instances,WMT:23-PCV-96163,2,2024-11-09 04:00:43.898,2024-11-08 14:20:20.365,node,23-PCV-96163,VRD - PH 1STSTG COMP DRY GAS SEAL SYSTEM,[],1686,1000000000229811,673287,1001,"{'space': 'springfield_instances', 'externalId..."
3,springfield_instances,WMT:23-TE-96114-01,1,2024-11-08 14:20:20.365,2024-11-08 14:20:20.365,node,23-TE-96114-01,VRD - PH 1STSTGGEAR 2 JOURNBRG DE,[],1686,1000000000682351,346423,8137,NaN
4,springfield_instances,WMT:23-HV-96191-03,1,2024-11-08 14:20:20.365,2024-11-08 14:20:20.365,node,23-HV-96191-03,VRD - PH 1STSTGLUBEOIL PSV OUT,[],1686,1000000000716882,346025,9802,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,springfield_instances,WMT:23-YXY-92526-03,1,2024-11-08 14:20:25.587,2024-11-08 14:20:25.587,node,23-YXY-92526-03,VRD - PH 1STSTGSUCTSCRUB B/D,[],1686,1000000000716643,347046,8137,NaN
96,springfield_instances,WMT:45-TT-92508,1,2024-11-08 14:20:25.587,2024-11-08 14:20:25.587,node,45-TT-92508,VRD - PH 1STSTGSUCTCLR COOLMED OUT,[],1686,1000000000710415,350362,9802,NaN
97,springfield_instances,WMT:23-FG-96180-01,1,2024-11-08 14:20:25.587,2024-11-08 14:20:25.587,node,23-FG-96180-01,VRD - PH 1STSTG LUBE OIL FILTER,[],1686,1000000000706034,345744,9802,NaN
98,springfield_instances,WMT:23-FY-92543,1,2024-11-08 14:20:25.587,2024-11-08 14:20:25.587,node,23-FY-92543,VRD - PH 1STSTGCOMP ANTI-SURGE,[],1686,1000000000705423,345889,8137,NaN


## Limitation with pagination

The search endpoint does not support pagination and can maximally return 1000 items. In the call above, we looked up all activites,
for 100 equiment, each which can have up to 1000 activites. The means that in the worse case we are getting only 1% of all the activites.
If we know that the each equipment has a lot of associated activites, or we want to ensure we are getting every single one back we can do
a lookup on the activites for one equipment at a time.

In [22]:
all_activities = dm.NodeList([])
for item in equipment:
    is_equipment = filters.ContainsAny(
        activity_view.as_property_ref("equipment"),
        [item.as_id()],
    )
    equipment_activity = client.data_modeling.instances.search(activity_view, filter=is_equipment, limit=1_000)
    all_activities.extend(equipment_activity)

In [23]:
all_activities

,space,external_id,version,last_updated_time,created_time,instance_type,assets,equipment,timeSeries,name,tags
0,springfield_instances,3285244535393803,3,2024-11-09 04:00:56.389,2024-11-08 14:20:28.693,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig ettersyn av instrumentering,[23-TE-96114-01]
1,springfield_instances,3914808270659650,3,2024-11-09 04:00:56.389,2024-11-08 14:20:31.073,node,[],"[{'space': 'springfield_instances', 'externalI...",[],,[23-TE-96114-01]
2,springfield_instances,124395033859817,3,2024-11-09 04:00:56.389,2024-11-08 14:20:31.073,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig sampling assessment,[23-TE-96114-01]
3,springfield_instances,3073969380122796,3,2024-11-09 04:00:56.389,2024-11-08 14:20:35.618,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig sampling assessment,[23-TE-96114-01]
4,springfield_instances,6742488065156238,3,2024-11-09 04:00:56.389,2024-11-08 14:20:36.244,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig ettersyn av instrumentering,[23-TE-96114-01]
...,...,...,...,...,...,...,...,...,...,...,...
1113,springfield_instances,2237100922212975,3,2024-11-09 04:01:18.514,2024-11-08 14:21:01.156,node,[],"[{'space': 'springfield_instances', 'externalI...",[],IPL/SIF - Funksjonstest av Sikkerhetskritisk I...,[23-FY-92543]
1114,springfield_instances,1102289785874474,3,2024-11-09 04:01:18.514,2024-11-08 14:21:01.904,node,[],"[{'space': 'springfield_instances', 'externalI...",[],Årlig ettersyn av instrumentering,[23-FY-92543]
1115,springfield_instances,1855708099224341,3,2024-11-09 04:01:18.514,2024-11-08 14:21:04.281,node,[],"[{'space': 'springfield_instances', 'externalI...",[],***-årlig visuell inspeksjon,[23-FY-92543]
1116,springfield_instances,1254254774950725,3,2024-11-09 04:01:18.514,2024-11-08 14:21:04.281,node,[],"[{'space': 'springfield_instances', 'externalI...",[],IPL/SIF - Funksjonstest av Sikkerhetskritisk I...,[23-FY-92543]


## Quering across edge

We will use thes `CogniteAnnotations` to find all assets connected to a file.


In [17]:
file_view = dm.ViewId("cdf_cdm", "CogniteFile", "v1")
annotation_view = dm.ViewId("cdf_cdm", "CogniteDiagramAnnotation", "v1")

In [18]:
query = dm.query.Query(
    with_={
        "file": dm.query.NodeResultSetExpression(
            limit=10,
            filter=filters.And(filters.HasData(views=[file_view]), filters.Equals(["node", "space"], data_space)),
        ),
        "annotation": dm.query.EdgeResultSetExpression(
            from_="file", direction="outwards", filter=filters.HasData(views=[annotation_view])
        ),
        "asset": dm.query.NodeResultSetExpression(from_="annotation", filter=filters.HasData(views=[asset_view])),
    },
    select={
        "file": dm.query.Select(sources=[dm.query.SourceSelector(source=file_view, properties=["*"])]),
        "annotation": dm.query.Select(sources=[dm.query.SourceSelector(source=annotation_view, properties=["*"])]),
        "asset": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
    },
)

result = client.data_modeling.instances.query(query)

In [19]:
result["file"]

,space,external_id,version,last_updated_time,created_time,instance_type,name,assets,mimeType,isUploaded,uploadedTime
0,springfield_instances,VAL_PH-25578-P-4110006-001.pdf,6,2024-11-09 12:59:09.104,2024-11-08 14:13:43.301,node,PH-25578-P-4110006-001.pdf,[],application/pdf,True,2024-11-09T12:59:08.963+00:00
1,springfield_instances,VAL_PH-25578-P-4110010-001.pdf,6,2024-11-09 12:59:09.252,2024-11-08 14:13:43.301,node,PH-25578-P-4110010-001.pdf,[],application/pdf,True,2024-11-09T12:59:09.221+00:00
2,springfield_instances,VAL_PH-25578-P-4110119-001.pdf,6,2024-11-09 12:59:09.521,2024-11-08 14:13:43.301,node,PH-25578-P-4110119-001.pdf,[],application/pdf,True,2024-11-09T12:59:09.493+00:00
3,springfield_instances,VAL_PH-ME-P-0003-001.pdf,6,2024-11-09 12:59:10.324,2024-11-08 14:13:43.301,node,PH-ME-P-0003-001.pdf,[],application/pdf,True,2024-11-09T12:59:09.781+00:00
4,springfield_instances,VAL_PH-ME-P-0004-001.pdf,6,2024-11-09 12:59:10.035,2024-11-08 14:13:43.301,node,PH-ME-P-0004-001.pdf,[],application/pdf,True,2024-11-09T12:59:10.007+00:00
5,springfield_instances,VAL_PH-ME-P-0151-001.pdf,6,2024-11-09 12:59:10.472,2024-11-08 14:13:43.301,node,PH-ME-P-0151-001.pdf,[],application/pdf,True,2024-11-09T12:59:10.446+00:00
6,springfield_instances,VAL_PH-ME-P-0152-001.pdf,6,2024-11-09 12:59:10.733,2024-11-08 14:13:43.301,node,PH-ME-P-0152-001.pdf,[],application/pdf,True,2024-11-09T12:59:10.707+00:00
7,springfield_instances,VAL_PH-ME-P-0153-001.pdf,6,2024-11-09 12:59:10.978,2024-11-08 14:13:43.301,node,PH-ME-P-0153-001.pdf,[],application/pdf,True,2024-11-09T12:59:10.947+00:00
8,springfield_instances,VAL_PH-ME-P-0156-001.pdf,6,2024-11-09 12:59:11.305,2024-11-08 14:13:43.301,node,PH-ME-P-0156-001.pdf,[],application/pdf,True,2024-11-09T12:59:11.277+00:00
9,springfield_instances,VAL_PH-ME-P-0156-002.pdf,6,2024-11-09 12:59:11.606,2024-11-08 14:13:43.301,node,PH-ME-P-0156-002.pdf,[],application/pdf,True,2024-11-09T12:59:11.562+00:00


In [20]:
result["annotation"].to_pandas().head()

,space,external_id,version,last_updated_time,created_time,instance_type,properties,type,start_node,end_node
0,springfield_instances,springfield_instances:VAL_PH-25578-P-4110006-0...,2,2024-11-09 13:19:46.786,2024-11-09 04:03:32.334,edge,{'cdf_cdm': {'CogniteDiagramAnnotation/v1': {'...,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i..."
1,springfield_instances,springfield_instances:VAL_PH-25578-P-4110006-0...,2,2024-11-09 13:19:46.786,2024-11-09 04:03:32.334,edge,{'cdf_cdm': {'CogniteDiagramAnnotation/v1': {'...,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i..."
2,springfield_instances,springfield_instances:VAL_PH-25578-P-4110006-0...,2,2024-11-09 13:19:46.786,2024-11-09 04:03:32.334,edge,{'cdf_cdm': {'CogniteDiagramAnnotation/v1': {'...,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i..."
3,springfield_instances,springfield_instances:VAL_PH-25578-P-4110006-0...,2,2024-11-09 13:19:46.786,2024-11-09 04:03:32.334,edge,{'cdf_cdm': {'CogniteDiagramAnnotation/v1': {'...,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i..."
4,springfield_instances,springfield_instances:VAL_PH-25578-P-4110006-0...,2,2024-11-09 13:19:46.786,2024-11-09 04:03:32.334,edge,{'cdf_cdm': {'CogniteDiagramAnnotation/v1': {'...,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i..."


In [21]:
result["asset"]

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,parent,pathLastUpdatedTime,name,description,sourceId,sourceUpdatedUser
0,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,4,2024-11-09 13:17:39.283,2024-11-08 14:20:28.103,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...","{'space': 'springfield_instances', 'externalId...",2024-11-09T13:17:39.28327+00:00,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,681825,8137
1,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,4,2024-11-09 13:17:39.283,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...","{'space': 'springfield_instances', 'externalId...",2024-11-09T13:17:39.28327+00:00,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,681824,8137
2,springfield_instances,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,2,2024-11-08 14:21:07.507,2024-11-08 14:21:06.876,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",NaN,2024-11-08T14:21:07.507048+00:00,23-1ST STAGE COMP LUBE OIL SYS-PH,1ST STAGE COMP LUBE OIL SYS ON PH,682544,8137


# Querying with pygen

`pygen` is a code generator that generates Python code tailored to your data model.

**Prerequisites**

* Installed `cognite-pygen`

In [22]:
from cognite.pygen import generate_sdk_notebook

In [23]:
pygen = generate_sdk_notebook(("cdf_cdm", "CogniteCore", "v1"), client, default_instance_space=data_space)

Successfully retrieved data model(s) ('cdf_cdm', 'CogniteCore', 'v1')
Cleaned temporary directory C:\Users\ANDERS~1\AppData\Local\Temp\pygen\cdf_cdm_CogniteCore_v1
Writing SDK to C:\Users\ANDERS~1\AppData\Local\Temp\pygen\cdf_cdm_CogniteCore_v1\cognite_core
Done!
Added C:\Users\ANDERS~1\AppData\Local\Temp\pygen\cdf_cdm_CogniteCore_v1 to sys.path to enable import
Imported cognite_core
You can now use the generated SDK in the current Python session.
The data classes are available by importing, for example, `from cognite_core.data_classes import CogniteDescribableWrite`


`pygen` enables you to use your data types diretly

In [24]:
assets = pygen.cognite_asset.list(space=data_space)

In [25]:
assets

,space,external_id,source_id,source_updated_user,name,description,parent,root,path,path_last_updated_time,data_record
0,springfield_instances,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,682546,8137,23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2ND STAGE COMP DRY GAS SEAL SYS ON PH,None,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,[WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH],2024-11-08 14:20:21.633609+00:00,"{'version': 2, 'last_updated_time': 2024-11-08..."
1,springfield_instances,WMT:230900,681760,8137,230900,GAS COMPRESSION AND RE-INJECTION ON PH,None,WMT:230900,[WMT:230900],2024-11-09 13:17:39.283270+00:00,"{'version': 3, 'last_updated_time': 2024-11-09..."
2,springfield_instances,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,682540,8137,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,None,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,[WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH],2024-11-08 14:20:28.094598+00:00,"{'version': 2, 'last_updated_time': 2024-11-08..."
3,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,681794,8137,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-2ND STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
4,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,681825,8137,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,WMT:23-2ND STAGE COMPRESSION-PH,WMT:230900,"[WMT:230900, WMT:23-2ND STAGE COMPRESSION-PH, ...",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
5,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,681824,8137,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,WMT:23-1ST STAGE COMPRESSION-PH,WMT:230900,"[WMT:230900, WMT:23-1ST STAGE COMPRESSION-PH, ...",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
6,springfield_instances,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,682544,8137,23-1ST STAGE COMP LUBE OIL SYS-PH,1ST STAGE COMP LUBE OIL SYS ON PH,None,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,[WMT:23-1ST STAGE COMP LUBE OIL SYS-PH],2024-11-08 14:21:07.507048+00:00,"{'version': 2, 'last_updated_time': 2024-11-08..."
7,springfield_instances,WMT:23-1ST STAGE COMP ENCLOSURE-PH,682542,8137,23-1ST STAGE COMP ENCLOSURE-PH,1ST STAGE COMP ENCLOSURE ON PH,None,WMT:23-1ST STAGE COMP ENCLOSURE-PH,[WMT:23-1ST STAGE COMP ENCLOSURE-PH],2024-11-08 14:21:07.507048+00:00,"{'version': 2, 'last_updated_time': 2024-11-08..."
8,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,681792,8137,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-1ST STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."


To get the `parent` and `children` as well as all other connections to each asset we can set the parameter `retrieve_connections='full'`

In [26]:
assets = pygen.cognite_asset.list(space=data_space, retrieve_connections="full")

In [28]:
assets

,space,external_id,source_id,source_updated_user,name,description,parent,root,path,path_last_updated_time,children,equipment,data_record
0,springfield_instances,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,682546,8137,23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2ND STAGE COMP DRY GAS SEAL SYS ON PH,None,"{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-08 14:20:21.633609+00:00,None,"[{'space': 'springfield_instances', 'external_...","{'version': 2, 'last_updated_time': 2024-11-08..."
1,springfield_instances,WMT:230900,681760,8137,230900,GAS COMPRESSION AND RE-INJECTION ON PH,None,"{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,"[{'space': 'springfield_instances', 'external_...","[{'space': 'springfield_instances', 'external_...","{'version': 3, 'last_updated_time': 2024-11-09..."
2,springfield_instances,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,682540,8137,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,None,"{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-08 14:20:28.094598+00:00,None,"[{'space': 'springfield_instances', 'external_...","{'version': 2, 'last_updated_time': 2024-11-08..."
3,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,681794,8137,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,"[{'space': 'springfield_instances', 'external_...",None,"{'version': 4, 'last_updated_time': 2024-11-09..."
4,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,681825,8137,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,None,"[{'space': 'springfield_instances', 'external_...","{'version': 4, 'last_updated_time': 2024-11-09..."
5,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,681824,8137,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,None,"[{'space': 'springfield_instances', 'external_...","{'version': 4, 'last_updated_time': 2024-11-09..."
6,springfield_instances,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,682544,8137,23-1ST STAGE COMP LUBE OIL SYS-PH,1ST STAGE COMP LUBE OIL SYS ON PH,None,"{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-08 14:21:07.507048+00:00,None,"[{'space': 'springfield_instances', 'external_...","{'version': 2, 'last_updated_time': 2024-11-08..."
7,springfield_instances,WMT:23-1ST STAGE COMP ENCLOSURE-PH,682542,8137,23-1ST STAGE COMP ENCLOSURE-PH,1ST STAGE COMP ENCLOSURE ON PH,None,"{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-08 14:21:07.507048+00:00,None,"[{'space': 'springfield_instances', 'external_...","{'version': 2, 'last_updated_time': 2024-11-08..."
8,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,681792,8137,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,"[{'space': 'springfield_instances', 'external_...","[{'space': 'springfield_instances', 'external_...","{'version': 4, 'last_updated_time': 2024-11-09..."


In [29]:
assets.parent

,space,external_id,source_id,source_updated_user,name,description,parent,root,path,path_last_updated_time,data_record
0,springfield_instances,WMT:230900,681760,8137,230900,GAS COMPRESSION AND RE-INJECTION ON PH,None,WMT:230900,[WMT:230900],2024-11-09 13:17:39.283270+00:00,"{'version': 3, 'last_updated_time': 2024-11-09..."
1,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,681794,8137,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-2ND STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
2,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,681792,8137,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-1ST STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
3,springfield_instances,WMT:230900,681760,8137,230900,GAS COMPRESSION AND RE-INJECTION ON PH,None,WMT:230900,[WMT:230900],2024-11-09 13:17:39.283270+00:00,"{'version': 3, 'last_updated_time': 2024-11-09..."


In [30]:
assets.children

,space,external_id,source_id,source_updated_user,name,description,parent,root,path,path_last_updated_time,data_record
0,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,681794,8137,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-2ND STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
1,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,681792,8137,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-1ST STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
2,springfield_instances,WMT:23-2ND STAGE COMPRESSOR-PH,681825,8137,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,WMT:23-2ND STAGE COMPRESSION-PH,WMT:230900,"[WMT:230900, WMT:23-2ND STAGE COMPRESSION-PH, ...",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
3,springfield_instances,WMT:23-1ST STAGE COMPRESSOR-PH,681824,8137,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,WMT:23-1ST STAGE COMPRESSION-PH,WMT:230900,"[WMT:230900, WMT:23-1ST STAGE COMPRESSION-PH, ...",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."


Similarily we can get the `CogniteEquipment`

In [43]:
equipments = pygen.cognite_equipment.list(space=data_space, retrieve_connections="full", limit=10)

In [44]:
equipments

,space,external_id,source_id,source_updated_user,name,description,asset,serial_number,manufacturer,files,activities,time_series,data_record
0,springfield_instances,WMT:23-YAH-96119-01,700491,1001,23-YAH-96119-01,VRD - PH 1STSTGGEAR 2 JOURNBRG NDE : VIBRATION...,None,1000000000293659,1686,[],None,None,"{'version': 1, 'last_updated_time': 2024-11-08..."
1,springfield_instances,WMT:23-TAH-96113-03,696113,1001,23-TAH-96113-03,VRD - PH 1STSTGGEAR 2 JOURNBRG NDE : TEMPERATU...,None,1000000000284937,1686,[],None,None,"{'version': 1, 'last_updated_time': 2024-11-08..."
2,springfield_instances,WMT:23-PCV-96163,673287,1001,23-PCV-96163,VRD - PH 1STSTG COMP DRY GAS SEAL SYSTEM,"{'space': 'springfield_instances', 'external_i...",1000000000229811,1686,[],None,None,"{'version': 2, 'last_updated_time': 2024-11-09..."
3,springfield_instances,WMT:23-TE-96114-01,346423,8137,23-TE-96114-01,VRD - PH 1STSTGGEAR 2 JOURNBRG DE,None,1000000000682351,1686,[],"[{'space': 'springfield_instances', 'external_...",None,"{'version': 1, 'last_updated_time': 2024-11-08..."
4,springfield_instances,WMT:23-HV-96191-03,346025,9802,23-HV-96191-03,VRD - PH 1STSTGLUBEOIL PSV OUT,None,1000000000716882,1686,[],"[{'space': 'springfield_instances', 'external_...",None,"{'version': 1, 'last_updated_time': 2024-11-08..."
5,springfield_instances,WMT:23-EZSH-92521,345680,4332,23-EZSH-92521,VRD - PH 1STSTGSUCTSCRUB LIQ OUT,None,1000000000708480,1686,[],"[{'space': 'springfield_instances', 'external_...",None,"{'version': 1, 'last_updated_time': 2024-11-08..."
6,springfield_instances,WMT:23-TAH-96116-03,770204,1001,23-TAH-96116-03,VRD - PH 1STSTGGEAR THRUST BRG OUT : TEMPERATU...,None,1000000000112739,1686,[],None,None,"{'version': 1, 'last_updated_time': 2024-11-08..."
7,springfield_instances,WMT:23-TE-96147,526509,1001,23-TE-96147,VRD - COMPRESSOR SEAL GAS HEATER TEMPERATURE,None,1000000000930230,1327,[],None,None,"{'version': 1, 'last_updated_time': 2024-11-08..."
8,springfield_instances,WMT:23-PDT-96167,650747,9802,23-PDT-96167,VRD - PH 1STSTG COMPR BAL PISTON LEAK,"{'space': 'springfield_instances', 'external_i...",1000000000179976,1686,[],"[{'space': 'springfield_instances', 'external_...","[{'space': 'springfield_instances', 'external_...","{'version': 2, 'last_updated_time': 2024-11-09..."
9,springfield_instances,WMT:23-PT-92531-01,696236,1001,23-PT-92531-01,SOFT TAG VRD - PH 1STSTGCOMP SUCTION,None,1000000000285061,1686,[],None,None,"{'version': 1, 'last_updated_time': 2024-11-08..."


In [45]:
equipments.asset

,space,external_id,source_id,source_updated_user,name,description,root,path,path_last_updated_time,data_record
0,springfield_instances,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,682540,8137,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,[WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH],2024-11-08 14:20:28.094598+00:00,"{'version': 2, 'last_updated_time': 2024-11-08..."
1,springfield_instances,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,682540,8137,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,[WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH],2024-11-08 14:20:28.094598+00:00,"{'version': 2, 'last_updated_time': 2024-11-08..."


In [46]:
equipments.activities

,space,external_id,name,tags,assets,equipment,time_series,data_record
0,springfield_instances,3285244535393803,Årlig ettersyn av instrumentering,[23-TE-96114-01],[],[WMT:23-TE-96114-01],[],"{'version': 3, 'last_updated_time': 2024-11-09..."
1,springfield_instances,3914808270659650,,[23-TE-96114-01],[],[WMT:23-TE-96114-01],[],"{'version': 3, 'last_updated_time': 2024-11-09..."
2,springfield_instances,124395033859817,Årlig sampling assessment,[23-TE-96114-01],[],[WMT:23-TE-96114-01],[],"{'version': 3, 'last_updated_time': 2024-11-09..."
3,springfield_instances,3073969380122796,Årlig sampling assessment,[23-TE-96114-01],[],[WMT:23-TE-96114-01],[],"{'version': 3, 'last_updated_time': 2024-11-09..."
4,springfield_instances,6742488065156238,Årlig ettersyn av instrumentering,[23-TE-96114-01],[],[WMT:23-TE-96114-01],[],"{'version': 3, 'last_updated_time': 2024-11-09..."
...,...,...,...,...,...,...,...,...
67,springfield_instances,3625599615521242,,[23-PDT-96167],[],[WMT:23-PDT-96167],"[pi:160649, pi:160649]","{'version': 5, 'last_updated_time': 2024-11-09..."
68,springfield_instances,3705032859825529,,[23-PDT-96167],[],[WMT:23-PDT-96167],"[pi:160649, pi:160649]","{'version': 5, 'last_updated_time': 2024-11-09..."
69,springfield_instances,6437204238499340,,[23-PDT-96167],[],[WMT:23-PDT-96167],"[pi:160649, pi:160649]","{'version': 5, 'last_updated_time': 2024-11-09..."
70,springfield_instances,2697963995334182,,[23-PDT-96167],[],[WMT:23-PDT-96167],"[pi:160649, pi:160649]","{'version': 5, 'last_updated_time': 2024-11-09..."


In addition, you can build up more complex querying with the `.query()` method in `pygen`.

In [60]:
children_from_parent = pygen.cognite_asset.query().name.equals("230900").children.list_cognite_asset()
children_from_parent

,space,external_id,source_id,source_updated_user,name,description,parent,root,path,path_last_updated_time,data_record
0,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,681794,8137,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-2ND STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
1,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,681792,8137,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,WMT:230900,WMT:230900,"[WMT:230900, WMT:23-1ST STAGE COMPRESSION-PH]",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."


In [61]:
children_by_parent = pygen.cognite_asset.query().parent.name.equals("230900").list_full(limit=-1)
children_by_parent

,space,external_id,source_id,source_updated_user,name,description,parent,root,path,path_last_updated_time,data_record
0,springfield_instances,WMT:23-2ND STAGE COMPRESSION-PH,681794,8137,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."
1,springfield_instances,WMT:23-1ST STAGE COMPRESSION-PH,681792,8137,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,"{'space': 'springfield_instances', 'external_i...","{'space': 'springfield_instances', 'external_i...","[{'space': 'springfield_instances', 'external_...",2024-11-09 13:17:39.283270+00:00,"{'version': 4, 'last_updated_time': 2024-11-09..."


In [65]:
activities = pygen.cognite_activity.query().equipment.external_id.equals("WMT:23-TE-96114-01").list_full(limit=-1)
activities

,space,external_id,name,tags,assets,equipment,time_series,data_record
0,springfield_instances,3285244535393803,Årlig ettersyn av instrumentering,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
1,springfield_instances,3914808270659650,,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
2,springfield_instances,124395033859817,Årlig sampling assessment,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
3,springfield_instances,3073969380122796,Årlig sampling assessment,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
4,springfield_instances,6742488065156238,Årlig ettersyn av instrumentering,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
5,springfield_instances,8933999992015645,Årlig sampling assessment,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
6,springfield_instances,3320010510817842,***-årlig visuell inspeksjon,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
7,springfield_instances,3668410190941194,Ettersyn av instrumentering,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
8,springfield_instances,4479477313749495,Ettersyn av instrumentering,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."
9,springfield_instances,5403672611002521,Ettersyn av instrumentering,[23-TE-96114-01],[],"[{'space': 'springfield_instances', 'external_...",[],"{'version': 3, 'last_updated_time': 2024-11-09..."


To learn more about `pygen` see the [Pygen docs](https://cognite-pygen.readthedocs-hosted.com/en/latest/)